In [ ]:
# standard imports
import torch
import torch.nn as nn
from sklearn.datasets import make_moons
from generate2d import sample2d, energy2d

# FrEIA imports
import FrEIA.framework as Ff
import FrEIA.modules as Fm

device = 'cuda' if torch.cuda.is_available() else 'cpu'

BATCHSIZE = 1000
N_DIM = 2

# we define a subnet for use inside an affine coupling block
# for more detailed information see the full tutorial
def subnet_fc(dims_in, dims_out):
    return nn.Sequential(nn.Linear(dims_in, 512), nn.ReLU(),
                         nn.Linear(512,  dims_out))

# a simple chain of operations is collected by ReversibleSequential
inn = Ff.SequenceINN(N_DIM)
for k in range(8):
    inn.append(Fm.AllInOneBlock, subnet_constructor=subnet_fc, permute_soft=True)

optimizer = torch.optim.Adam(inn.parameters(), lr=0.001)

In [ ]:
def plot_density2d(model, args, limit=4, step=0.01):

    grid = torch.Tensor(
        [
            [a, b]
            for a in np.arange(-limit, limit, step)
            for b in np.arange(-limit, limit, step)
        ]
    )
    grid_dataset = torch.utils.data.TensorDataset(grid.to(args.device))
    grid_data_loader = torch.utils.data.DataLoader(
        grid_dataset, batch_size=10000, shuffle=False
    )

    prob = torch.cat(
        [
            torch.exp(compute_log_p_x(model, x_mb)).detach()
            for x_mb, in grid_data_loader
        ],
        0,
    )

    prob = prob.view(int(2 * limit / step), int(2 * limit / step)).t()

    if args.reduce_extreme:
        prob = prob.clamp(max=prob.mean() + 3 * prob.std())

    plt.figure(figsize=(12, 12))
    plt.imshow(prob.cpu().data.numpy(), extent=(-limit, limit, -limit, limit))
    plt.axis("off")
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1)

    if args.savefig:
        plt.savefig(
            os.path.join(
                args.load or args.path, "{}.jpg".format(datetime.datetime.now())
            )
        )
    else:
        plt.show()

In [ ]:
# a very basic training loop
for i in range(10000):
    
    optimizer.zero_grad()
    # sample data from the moons distribution
#     data, label = make_moons(n_samples=BATCHSIZE, noise=0.05)
    data = sample2d('8gaussians', BATCHSIZE)
    x = torch.Tensor(data)
    # pass to INN and get transformed variable z and log Jacobian determinant
    z, log_jac_det = inn(x)
    # calculate the negative log-likelihood of the model with a standard normal prior
    loss = 0.5*torch.sum(z**2, 1) - log_jac_det
    loss = loss.mean() / N_DIM
    # backpropagate and update the weights
    loss.backward()
    optimizer.step()
    
